In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
df = pd.read_csv('main_task.xls')
df.isna().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [2]:
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [3]:
# удалим столбцы, которые точно не понадобятся
df=df.drop(['URL_TA','ID_TA'], axis = 1)
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ..."
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']..."
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],..."
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]"
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b..."


# Reviews

In [4]:
# заменяем пропуски на 0
df['Number of Reviews'] = df['Number of Reviews'].fillna(0)

In [5]:
# признак наличия/отсутствия отзыва

def Y_N(row):
    if row>0:
        row=1
    else:
        row
    return row

df['ReviewY/N'] = df['Number of Reviews'].apply(Y_N)
df['ReviewY/N'].value_counts()

1.0    37457
0.0     2543
Name: ReviewY/N, dtype: int64

In [6]:
def Review_opt(row):
    '''удаляем ненужные символы и формируем список в Rewievs'''
    row = row.replace('[[','')\
    .replace(']]','')\
    .split('], [')
    return row

def date_delta(row):
    '''определяем количество дней между отзывами'''
    row = row[1][1:-1].split("', '")
    if len(row)==2:
        row = pd.to_datetime(row)
        row = abs((row[0]-row[1]).days)
    else:
        row = 0
    return row

def only_dates(row):
    '''выведем только даты последних отзывов'''
    row = row[1][1:-1].split("', '")
    if len(row) > 1:
        row = max(pd.to_datetime(row))
    else:
        row = pd.to_datetime(row)     #row = abs((row[0]-row[1]).days)
    
    return row

In [7]:
rew_opt = df['Reviews'].apply(Review_opt) # оптимизация столбца

dates_delta = rew_opt.apply(date_delta) # разница в днях между отзывами


In [8]:
# скорее всего частота написания отзывов находится в обратной зависимости с рейтингом
def revers(x):
    if x>0:
        x=1/x
    else:
        x=0
    return x

df['Review_freq'] = dates_delta.apply(revers)

In [9]:
# выведем признак свежести отзыва по сравнению с самым последним в df
only_last_dates = rew_opt.apply(only_dates) #только последние даты
last_date = only_last_dates.explode().max() # дата последнего отзыва
diff_date = only_last_dates.apply(lambda x: (last_date - x).days)

def rev(x):
    '''обратим число с проверкой по типу'''
    if type(x) is int and x!=0:
        x = 1/x
    else:
        x = 0
    return x

df['Review_last'] = diff_date.apply(lambda x: rev(x))

In [28]:
only_last_dates.explode().max()

Timestamp('2018-02-26 00:00:00')

# Cuisine Style

In [10]:
# преобразуем данные в список

df['Cuisine Style'] = df['Cuisine Style'].dropna().apply(lambda x: x[2:-2].split("', '"))

In [11]:
common_cuisine = df['Cuisine Style'].explode().value_counts(normalize = True).head(26).round(4)*100
common_cuisine

Vegetarian Friendly    11.70
European               10.52
Mediterranean           6.57
Italian                 6.24
Vegan Options           4.69
Gluten Free Options     4.30
Bar                     3.45
French                  3.34
Asian                   3.15
Pizza                   2.98
Spanish                 2.93
Pub                     2.56
Cafe                    2.43
Fast Food               1.78
British                 1.67
International           1.66
Seafood                 1.57
Japanese                1.53
Central European        1.46
American                1.38
Sushi                   1.21
Chinese                 1.20
Portuguese              1.16
Indian                  1.09
Middle Eastern          0.82
Thai                    0.78
Name: Cuisine Style, dtype: float64

In [12]:
most_common = ['Vegetarian Friendly','European']

def com_not_com(row):
    if most_common[0] in row or most_common[1] in row:
        row = 1
    else:
        row = 0
    return row

In [13]:
# создадим признак количества предлагаемых вариантов кухонь
df['Amount_of_Cuisine'] = df['Cuisine Style'].dropna().apply(lambda x: len(x))

In [14]:
# найдем срединое количество кухонь в ресторанах по городам
Cuisine_median = df.groupby('City')['Amount_of_Cuisine'].median().round()


In [15]:
# заполним пропуски в соответствии со средним значением по городу
def fill_cuisine(row):
    
    if pd.isna(row.Amount_of_Cuisine) == True:
        row.Amount_of_Cuisine = Cuisine_median[row.City]
    return row

df = df.apply(lambda row: fill_cuisine(row), axis=1)

In [16]:
# определим функцию заполнения пропусков  в 'Cuisine Style' наиболее частыми значениями в данном городе  

City = df.City.value_counts().index
def fill_cuis(row):
    if type(row['Cuisine Style']) == float:
        s = int(Cuisine_median[row.City])
        row['Cuisine Style'] = list(df[df['City'] == row.City]['Cuisine Style'].explode().value_counts().index[:s])
    return row

In [17]:
df = df.apply(lambda row: fill_cuis(row), axis=1)

In [18]:
# признак наличия популярных видов кухонь
df['Comm_Cuisine Style'] = df['Cuisine Style'].dropna().apply(com_not_com)

# Price Range

In [19]:
# замена на числовое выражение вариантов цен (пропуски на среднее значение)
Price_int = {'$':1, '$$ - $$$':2,'$$$$':3, np.NaN:2 }
df['Price_int'] = df['Price Range'].map(Price_int)

# City

In [20]:
dummies = pd.get_dummies(df.City)
df = pd.concat([df, dummies], axis=1)

# ML

In [21]:
df_ml = df.drop(['Restaurant_id','City','Cuisine Style','Price Range','Reviews'],axis=1)

In [22]:
df_ml.head()

,Ranking,Rating,Number of Reviews,ReviewY/N,Review_freq,Review_last,Amount_of_Cuisine,Comm_Cuisine Style,Price_int,Amsterdam,...,Munich,Oporto,Oslo,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
0,5570.0,3.5,194.0,1.0,0.024390,0.017544,3.0,1,2,0,...,0,0,0,1,0,0,0,0,0,0
1,1537.0,4.0,10.0,1.0,0.002618,0.004255,3.0,1,2,0,...,0,0,0,0,0,0,1,0,0,0
2,353.0,4.5,688.0,1.0,0.500000,0.020408,7.0,1,3,0,...,0,0,0,0,0,0,0,0,0,0
3,3458.0,5.0,3.0,1.0,0.000000,0.000000,3.0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
4,621.0,4.0,84.0,1.0,0.003676,0.010000,3.0,1,2,0,...,1,0,0,0,0,0,0,0,0,0


In [23]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
    # Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
x = df_ml.drop([ 'Rating'], axis = 1)  
y = df_ml['Rating']  
      
    # Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
      
    # Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
    # Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [24]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
      
    # Создаём модель  
regr = RandomForestRegressor(n_estimators=100)        
    # Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
    # Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

In [25]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  

MAE: 0.21107949999999998
